In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time

In [2]:
# Scrape website
# html=browser.html

# Find all the car listings on the page
# car_list=car_soup.find_all('div', class_='vehicle-card')

# Testing scraping for the first listing on the site
# car_model=car_soup.find('h2', class_='title')
# car_status=car_soup.find('p', class_='stock-type')
# car_price=car_soup.find('span', class_='primary-price')
# car_dealer=car_soup.find('div', class_='dealer-name')
# dealer_city=car_soup.find('div', class_='miles-from')

# print(car_model, car_price, car_dealer, car_status, dealer_city)


In [3]:
# Testing to make sure text that is stripped is correct
#model=car_model.get_text(strip=True)
#status=car_status.get_text(strip=True)
#price=car_price.get_text(strip=True)
#dealer=car_dealer.get_text(strip=True)
#city=dealer_city.get_text(strip=True)

#print(model, status, price, dealer, city)

In [4]:
# Set up Splinter
browser = Browser('chrome')

# Base URL and initial page
base_url = 'https://www.cars.com/shopping/results/?makes[]=subaru&maximum_distance=150&models[]=subaru-crosstrek&page={}&page_size=100&stock_type=all&zip=61606'
page_number = 1
url = base_url.format(page_number)

# Create lists to store details
models = []
statuses = []
mileages = []
prices = []
dealers = []
cities = []
urls = []  # List to store URLs

# Set to store unique identifiers of scraped cars
scraped_car_ids = set()

# Function to scrape current page
def scrape_page(url):
    browser.visit(url)
    time.sleep(5)  # Adjust as necessary
    
    html = browser.html
    car_soup = soup(html, 'html.parser')
    car_list = car_soup.find_all('div', class_='vehicle-card')

    if not car_list:
        print("No more car listings found. Exiting.")
        return False

    for car in car_list:
        # Extract car model
        car_model = car.find('h2', class_='title')
        model = str(car_model.get_text(strip=True)) if car_model else 'N/A'

        # Extract car mileage
        car_mileage = car.find('div', class_='mileage')
        mileage = str(car_mileage.get_text(strip=True)) if car_mileage else 'N/A'

        # Extract car status
        car_status = car.find('p', class_='stock-type')
        status = str(car_status.get_text(strip=True)) if car_status else 'N/A'

        # Extract car price
        car_price = car.find('span', class_='primary-price')
        price = str(car_price.get_text(strip=True)) if car_price else 'N/A'

        # Extract dealer name
        car_dealer_div = car.find('div', class_='dealer-name')
        if car_dealer_div:
            car_dealer = car_dealer_div.find('strong')
            dealer = str(car_dealer.get_text(strip=True)) if car_dealer else 'N/A'
        else:
            dealer = 'N/A'

        # Extract dealer city
        dealer_city = car.find('div', class_='miles-from')
        city = str(dealer_city.get_text(strip=True)) if dealer_city else 'N/A'

        # Extract car URL
        car_url = car.find('a', class_='vehicle-card-link')['href'] if car.find('a', class_='vehicle-card-link') else 'N/A'

        # Check if this car has already been scraped
        if car_url in scraped_car_ids:
            continue  # Skip if already scraped
        
        # Add car URL to set of scraped car IDs
        scraped_car_ids.add(car_url)

        # Append details to lists
        models.append(model)
        statuses.append(status)
        prices.append(price)
        dealers.append(dealer)
        cities.append(city)
        mileages.append(mileage)
        urls.append(car_url)  # Append URL to list

    return True

# Counter to track consecutive pages with no new data
no_new_data_count = 0
max_no_new_data_allowed = 3  # Adjust as necessary

# Timeout settings
timeout_seconds = 240  # 4 minutes timeout
start_time = time.time()

# Loop through pages
while True:
    if time.time() - start_time > timeout_seconds:
        print(f"Timeout ({timeout_seconds} seconds) exceeded. Exiting.")
        break
    
    if not scrape_page(url):
        # Increment no_new_data_count if no new data found
        no_new_data_count += 1
        if no_new_data_count >= max_no_new_data_allowed:
            print(f"No new data found for {max_no_new_data_allowed} consecutive pages. Exiting.")
            break
    else:
        # Reset no_new_data_count if new data found
        no_new_data_count = 0
    
    # Increment page number and update URL
    page_number += 1
    url = base_url.format(page_number)
    print(f"Scraping page {page_number}...")

# Print number of cars scraped
print(f"Number of unique cars scraped: {len(scraped_car_ids)}")

Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Timeout (240 seconds) exceeded. Exiting.
Number of unique cars scraped: 921


In [5]:
# Create df
data={
    'Model': models,
    'Mileage (mi)': mileages,
    'Status': statuses,
    'Price (USD)': prices,
    'Dealer': dealers,
    'City': cities ,
    'URL': urls
    }
car_df= pd.DataFrame(data)

In [6]:
car_df

,Model,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2024 Subaru Crosstrek Wilderness,N/A,New,"$37,064",Uftring Ford Subaru,"East Peoria, IL",/vehicledetail/cf66a4e7-6222-4971-be1f-1fe3807...
1,2024 Subaru Crosstrek Sport,N/A,New,"$31,310",Uftring Ford Subaru,"East Peoria, IL (5 mi.)",/vehicledetail/c917d37e-4b4b-4c99-8aed-e7081fc...
2,2024 Subaru Crosstrek Limited,N/A,New,"$35,080",Uftring Ford Subaru,"East Peoria, IL (5 mi.)",/vehicledetail/9555705e-0fc4-4a07-99e4-61086b5...
3,2021 Subaru Crosstrek Limited,"42,835 mi.",Used,"$25,967",Uftring Ford Subaru,"East Peoria, IL (5 mi.)",/vehicledetail/c0e9fd3f-fc8e-4fd6-9875-8ed94a3...
4,2021 Subaru Crosstrek Premium,"41,685 mi.",Used,"$23,998",CarMax Austin North - Offering Express Pickup ...,N/A,/vehicledetail/4fd8f145-9ac9-4ead-98f0-0cdbab2...
...,...,...,...,...,...,...,...
916,2018 Subaru Crosstrek 2.0i Premium,"83,166 mi.",Used,"$18,988",Stinnett Chevrolet Buick Chrysler Jeep Dodge RAM,"Newport, TN (476 mi.)",/vehicledetail/a88cfc6d-39f5-493d-8fbe-e23dc01...
917,2018 Subaru Crosstrek 2.0i Limited,"159,450 mi.",Used,"$14,827",Midtown Kia,"Tulsa, OK (467 mi.)",/vehicledetail/9b84fb75-9d02-4fc9-875b-0dcf7ac...
918,2023 Subaru Crosstrek Limited,"39,758 mi.",Used,"$26,728",My Auto Import Center,"Muskegon, MI (244 mi.)",/vehicledetail/85e19e3b-173d-4aa2-be16-d77a49c...
919,2021 Subaru Crosstrek Limited,"30,178 mi.",Used,"$26,115",Reineke Ford Lincoln,"Findlay, OH (313 mi.)",/vehicledetail/80cbc6ab-5642-4982-a894-694a5dc...


In [7]:
# Checking to make sure df is formatted correctly
# print(car_df.to_string())

In [8]:
# Close the browser
browser.quit()

In [9]:
# Display DF
print("Original DataFrame:")
print(car_df)
print()

Original DataFrame:
                                  Model Mileage (mi) Status Price (USD)  \
0      2024 Subaru Crosstrek Wilderness          N/A    New     $37,064   
1           2024 Subaru Crosstrek Sport          N/A    New     $31,310   
2         2024 Subaru Crosstrek Limited          N/A    New     $35,080   
3         2021 Subaru Crosstrek Limited   42,835 mi.   Used     $25,967   
4         2021 Subaru Crosstrek Premium   41,685 mi.   Used     $23,998   
..                                  ...          ...    ...         ...   
916  2018 Subaru Crosstrek 2.0i Premium   83,166 mi.   Used     $18,988   
917  2018 Subaru Crosstrek 2.0i Limited  159,450 mi.   Used     $14,827   
918       2023 Subaru Crosstrek Limited   39,758 mi.   Used     $26,728   
919       2021 Subaru Crosstrek Limited   30,178 mi.   Used     $26,115   
920  2016 Subaru Crosstrek 2.0i Premium  131,588 mi.   Used     $11,861   

                                                Dealer  \
0                    

In [10]:
# Check for missing values:
missing_values= car_df.isnull().sum()
print("Missing values:")
print(missing_values)
print()

Missing values:
Model           0
Mileage (mi)    0
Status          0
Price (USD)     0
Dealer          0
City            0
URL             0
dtype: int64



In [11]:
# Split 'Model' column into 'Year', 'Manufacturer', 'Model', 'Trim'
car_df[['Year', 'Manufacturer', 'Model', 'Trim']] = car_df['Model'].str.split(' ', 3, expand=True)

# Replace 'Not Priced' with NaN and convert 'Price' column to numeric
car_df['Price (USD)'] = pd.to_numeric(car_df['Price (USD)'].replace('Not Priced', pd.NA).str.replace('[\$,]', '', regex=True), errors='coerce')

# Clean Mileage column
car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')

# Reorder columns as required
car_df = car_df[['Year', 'Manufacturer', 'Model', 'Trim', 'Mileage (mi)', 'Status', 'Price (USD)', 'Dealer', 'City', 'URL']]

# Display the updated DataFrame
print("Final DataFrame:")
print(car_df)

Final DataFrame:
     Year Manufacturer      Model          Trim  Mileage (mi) Status  \
0    2024       Subaru  Crosstrek    Wilderness           NaN    New   
1    2024       Subaru  Crosstrek         Sport           NaN    New   
2    2024       Subaru  Crosstrek       Limited           NaN    New   
3    2021       Subaru  Crosstrek       Limited       42835.0   Used   
4    2021       Subaru  Crosstrek       Premium       41685.0   Used   
..    ...          ...        ...           ...           ...    ...   
916  2018       Subaru  Crosstrek  2.0i Premium       83166.0   Used   
917  2018       Subaru  Crosstrek  2.0i Limited      159450.0   Used   
918  2023       Subaru  Crosstrek       Limited       39758.0   Used   
919  2021       Subaru  Crosstrek       Limited       30178.0   Used   
920  2016       Subaru  Crosstrek  2.0i Premium      131588.0   Used   

     Price (USD)                                             Dealer  \
0        37064.0                               

C:\Users\awsom\AppData\Local\Temp\ipykernel_4504\2381806475.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')


In [13]:
car_df

,Year,Manufacturer,Model,Trim,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2024,Subaru,Crosstrek,Wilderness,NaN,New,37064.0,Uftring Ford Subaru,"East Peoria, IL",/vehicledetail/cf66a4e7-6222-4971-be1f-1fe3807...
1,2024,Subaru,Crosstrek,Sport,NaN,New,31310.0,Uftring Ford Subaru,"East Peoria, IL (5 mi.)",/vehicledetail/c917d37e-4b4b-4c99-8aed-e7081fc...
2,2024,Subaru,Crosstrek,Limited,NaN,New,35080.0,Uftring Ford Subaru,"East Peoria, IL (5 mi.)",/vehicledetail/9555705e-0fc4-4a07-99e4-61086b5...
3,2021,Subaru,Crosstrek,Limited,42835.0,Used,25967.0,Uftring Ford Subaru,"East Peoria, IL (5 mi.)",/vehicledetail/c0e9fd3f-fc8e-4fd6-9875-8ed94a3...
4,2021,Subaru,Crosstrek,Premium,41685.0,Used,23998.0,CarMax Austin North - Offering Express Pickup ...,N/A,/vehicledetail/4fd8f145-9ac9-4ead-98f0-0cdbab2...
...,...,...,...,...,...,...,...,...,...,...
916,2018,Subaru,Crosstrek,2.0i Premium,83166.0,Used,18988.0,Stinnett Chevrolet Buick Chrysler Jeep Dodge RAM,"Newport, TN (476 mi.)",/vehicledetail/a88cfc6d-39f5-493d-8fbe-e23dc01...
917,2018,Subaru,Crosstrek,2.0i Limited,159450.0,Used,14827.0,Midtown Kia,"Tulsa, OK (467 mi.)",/vehicledetail/9b84fb75-9d02-4fc9-875b-0dcf7ac...
918,2023,Subaru,Crosstrek,Limited,39758.0,Used,26728.0,My Auto Import Center,"Muskegon, MI (244 mi.)",/vehicledetail/85e19e3b-173d-4aa2-be16-d77a49c...
919,2021,Subaru,Crosstrek,Limited,30178.0,Used,26115.0,Reineke Ford Lincoln,"Findlay, OH (313 mi.)",/vehicledetail/80cbc6ab-5642-4982-a894-694a5dc...


In [10]:
#print(car_df.to_string())

In [14]:
car_df.to_excel("Crosstrek.xlsx")